In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import timm

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
from tqdm.notebook import tqdm

In [11]:
class artStylesDataset(Dataset):
    def __init__(self,data_dir,transform=None):
        self.data = ImageFolder(data_dir, transform=transform)

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        return self.data[idx]

    @property
    def classes(self):
        return self.data.classes

In [19]:
target_to_class = {v: k for k, v in ImageFolder("./Data").class_to_idx.items()}
print(target_to_class)

{0: 'baroque', 1: 'cubism', 2: 'pop-art', 3: 'postimpressionism'}


In [25]:
transform = transforms.Compose([
    transforms.Resize((464,600)),
    transforms.ToTensor(),
])

dataset = artStylesDataset("./Data",transform)

In [26]:
for image, label in dataset:
    break

In [27]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [29]:
for images, labels in dataloader:
    break